<a href="https://colab.research.google.com/github/MaheepChaudhary/Prototypical_Network_Alphabet_Recognition/blob/master/Prototypical_Network_Alphabet_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unrar x '/content/drive/My Drive/meta_learning/data.rar'

Streaming output truncated to the last 5000 lines.
Extracting  data/images/Sylheti/character02/1504_11.png                   84%  OK 
Extracting  data/images/Sylheti/character02/1504_12.png                   84%  OK 
Extracting  data/images/Sylheti/character02/1504_13.png                   84%  OK 
Extracting  data/images/Sylheti/character02/1504_14.png                   85%  OK 
Extracting  data/images/Sylheti/character02/1504_15.png                   85%  OK 
Extracting  data/images/Sylheti/character02/1504_16.png                   85%  OK 
Extracting  data/images/Sylheti/character02/1504_17.png                   85%  OK 
Extracting  data/images/Sylheti/character02/1504_18.png                   85%  OK 
Extracting  data/images/Sylheti/character02/1504_19.png                   85%  OK 
Extracting  data/images/Sylheti/character02/1504_20.png                   85%  OK 
Creating    data/images/Sylhe

Importing the necessary libraries

In [ ]:
import os
import tensorflow.keras as tf
import numpy as np
from tensorflow.keras.layers import Dense,Conv2D,Activation,Input,MaxPool2D,Dropout,Lambda
from pprint import pprint
from PIL import Image
from tqdm import tqdm
from keras import backend as K 

Preprocessing the Data

In [ ]:
#accesisng the classes given in train.txt file
root_dir = '/content/data/'
train_split_path = os.path.join(root_dir, 'splits', 'train.txt')

with open(train_split_path, 'r') as train_split:
  train_classes = [line.rstrip() for line in train_split.readlines()]

no_of_classes = len(train_classes) 
print(no_of_classes)   

4112


In [ ]:
#creating a list i.e. all_images of size [no_of_classes,no_of_examples,width_of_image,height_of_image]
all_images = []
h_width,h_height = 28,28

for clas in tqdm(train_classes):
  cls,character,rotation =  str(clas).split('/')
  rotation = float(rotation[3:])
  cls_images = []
  for img in os.listdir(os.path.join(root_dir,'images',cls,character)):
    cls_images.append(1. - np.array(Image.open(os.path.join(root_dir,'images',cls,character,img)).rotate(rotation).resize((h_width,h_height))))
  all_images.append(cls_images)

100%|██████████| 4112/4112 [00:17<00:00, 230.92it/s]


Building the embedding model

In [ ]:
def embedding(input_shape):
  input_layer = Input(input_shape)
  layer_1 = Conv2D(4*2,input_shape=input_shape,kernel_size=1,activation='relu')(input_layer)
  layer_1_1 = MaxPool2D(pool_size=(2,2),strides=1)(layer_1)
  layer_2 = Conv2D(8*2,kernel_size=3,activation='relu')(layer_1_1)
  layer_2_2 = MaxPool2D(pool_size=(2,2),strides=2)(layer_2)
  layer_drop = Dropout(0.1)(layer_2_2)
  layer_3 = Conv2D(16*2,kernel_size=3,activation='relu')(layer_drop)
  layer_3_3 = MaxPool2D(pool_size=(2,2),strides=2)(layer_3)
  layer_drop_1 = Dropout(0.2)(layer_3_3)
  layer_4 = Conv2D(128,kernel_size=3,activation='relu')(layer_3_3)
  layer_4_4 = MaxPool2D(pool_size=(2,2),strides=1)(layer_4)
  layer_drop_2 = Dropout(0.3)(layer_4_4)
  model = tf.Model(inputs=[input_layer],outputs=[layer_drop_2]) 
  return model

In [ ]:
import numpy as np
from pprint import pprint

def euclidean(vect):
  x,y = vect
  distance = K.sqrt(K.sum(K.square(x-y),axis = 0))
  return distance

def mean(arr):
  result = arr
  for i in range(0,len(arr)-1):
    result = np.add(arr[i+1],result)
  print(np.array(result).shape)
  return (np.divide(result,len(arr)).reshape(h_width,h_height,1))
  
mean()  

In [ ]:
def softmax(no_of_classes):
  pred = tf.nn.softmax(len(no_of_calsses))

In [ ]:
input_shape = (h_width,h_height,1)
input_proto = Input(shape = input_shape)
input_diff = Input(shape = input_shape)

mod = embedding(input_shape)
input_1 = mod(input_proto)
input_2 = mod(input_diff)

output = Lambda(euclidean)([input_proto,input_diff])
pred = Dense(no_of_classes,activation='softmax')(output)
prototypical_model = tf.Model(inputs = [input_proto,input_diff],outputs = pred)


Compiling the model

In [ ]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse = False)

arr = [0,2,4,3,2,4]
arr = np.array(arr).reshape((len(arr),1))
arr = onehot_encoder.fit_transform(arr)
print(arr)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]


In [ ]:
import random
from random import seed
from random import randint
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse = False)
epochs = 30
episodes = 100
n_way = 60
n_shot = 5
n_query = 5
n_examples = 20
query_batch,mean_batch,label,prototype_of_classes = [],[],[],[]

for i in tqdm(range(epochs)):
  #doing episodic training
  for i in range(episodes):
    selected_classes = []
    seed(i)
    #taking the random numbers for selecting classes 
    while(True):
      selected_classes.append(randint(0,no_of_classes-1))
      if len(selected_classes)>=n_way:break
    #taking the random numbers for selecting images
    for f in selected_classes:
      seed(f)
      full_batch = []
      selected_images = []
      while(True):
        selected_images.append(randint(0,n_examples-1))
        if len(selected_images)>=(n_shot+n_query):break
      #selecting images randomly
      for k in selected_images:   
        full_batch.append(all_images[f][k])
      query_batch.append(full_batch[:n_shot])
      mean_batch.append([full_batch[n_shot:]])
      label.append(f)
    #converting the label to onehot_encoding
    #label = np.array(label).reshape(len(label),1)
    #x = [prototype_of_classes,query_batch],y = label
    #label = onehot_encoder.fit_transform(label)
    prototype_of_classes = [mean(x) for x in mean_batch]
    temp = list(zip(query_batch,label))
    random.shuffle(temp)
    print(temp)
    query_batch,label = zip(*temp)
    prototypical_model.compile(optimizer = 'adam',loss = 'mse',metrics = ['mse'])
    history = prototypical_model.fit([np.array(prototype_of_classes),np.array(query_batch)],np.array(label)) 


  0%|          | 0/30 [00:00<?, ?it/s]

(1, 5, 28, 28)


ValueError: ignored

In [ ]:
print(np.array(prototype_of_classes).shape)
print(np.array(query_batch).shape)

(60, 1, 5, 28, 28)
(60, 5, 28, 28)


In [ ]:
all_images[331][19]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0